In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
!pip install psycopg2-binary -q
import psycopg2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 25.4 MB/s eta 0:00:00


In [54]:
import pandas as pd
import sqlite3

In [55]:
# Cargar el archivo Excel desde Google Drive
archivo = '/content/drive/MyDrive/Prueba_tecnica_tuya/bd.xlsx'
xls = pd.ExcelFile(archivo)

In [56]:

# Crear una conexión con la base de datos SQLite (se crea si no existe)
conn = sqlite3.connect('/content/drive/MyDrive/Prueba_tecnica_tuya/base_de_datos.db')

# Crear un cursor para ejecutar comandos SQL
cursor = conn.cursor()

In [58]:
# Crear tablas (si es necesario)
cursor.execute('''
CREATE TABLE IF NOT EXISTS clientes (
    IDENTIFICACION TEXT,
    NOMBRE TEXT,
    TIPO_DOCUMENTO TEXT,
    CLASIFICACION TEXT,
    TIPO_TARJETA TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS transacciones (
    IDENTIFICACION TEXT,
    CODIGO_CATEGORIA TEXT,
    FECHA_TRANSACCION TEXT,
    ESTADO TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS categorias (
    CODIGO_CATEGORIA TEXT,
    NOMBRE_CATEGORIA TEXT
)
''')


In [60]:
# Cargar los DataFrames desde el archivo Excel
archivo = '/content/drive/MyDrive/Prueba_tecnica_tuya/bd.xlsx'
xls = pd.ExcelFile(archivo)

df_clientes = xls.parse('CLIENTES')
df_transacciones = xls.parse('TRANSACCIONES')
df_categorias = xls.parse('CATEGORIAS_CONSUMO')

In [61]:
# Limpieza de fechas en transacciones
df_transacciones['FECHA_TRANSACCION'] = pd.to_datetime(
    df_transacciones['FECHA_TRANSACCION'],
    errors='coerce',
    format='mixed'
)

# Reemplazar fechas inválidas por un valor neutral (opcional pero evita errores)
df_transacciones['FECHA_TRANSACCION'].fillna(pd.to_datetime("1900-01-01"), inplace=True)


<ipython-input-61-8b8c641d1956>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_transacciones['FECHA_TRANSACCION'].fillna(pd.to_datetime("1900-01-01"), inplace=True)


In [62]:
# Insertar datos en las tablas (si es necesario)
df_clientes.to_sql('clientes', conn, if_exists='replace', index=False)
df_transacciones.to_sql('transacciones', conn, if_exists='replace', index=False)
df_categorias.to_sql('categorias', conn, if_exists='replace', index=False)

60

In [66]:
df_clientes.columns

Index(['NOMBRE', 'IDENTIFICACIÓN', 'TIPO_DOCUMENTO', 'CLASIFICACION',
       'TIPO TARJETA', 'FEÇHA_APERTURA_TARJETA', 'ESTADO_TARJETA'],
      dtype='object')

In [63]:
# Parámetros personalizables
fecha_inicio = "2020-01-01"
fecha_fin = "2025-12-31"
top_n = 3  # Cambia esto a 2 o 3 si quieres más niveles de preferencia


In [71]:
# Consulta SQL
query = f"""
WITH trans_filtradas AS (
    SELECT
        *
    FROM transacciones
    WHERE
        FECHA_TRANSACCION BETWEEN '{fecha_inicio}' AND '{fecha_fin}'
        AND ESTADO = 'Aprobada'
),
conteo_transacciones AS (
    SELECT
        t.IDENTIFICACION,
        t.CODIGO_CATEGORIA,
        COUNT(*) AS num_transacciones,
        MAX(t.FECHA_TRANSACCION) AS ultima_fecha
    FROM trans_filtradas t
    GROUP BY t.IDENTIFICACION, t.CODIGO_CATEGORIA
),
ranking AS (
    SELECT
        c.IDENTIFICACION,
        cat.NOMBRE_CATEGORIA,
        c.num_transacciones,
        c.ultima_fecha,
        ROW_NUMBER() OVER (
            PARTITION BY c.IDENTIFICACION
            ORDER BY c.num_transacciones DESC, c.ultima_fecha DESC
        ) AS rk
    FROM conteo_transacciones c
    LEFT JOIN categorias cat
        ON c.CODIGO_CATEGORIA = cat.CODIGO_CATEGORIA
),
preferencias_top_n AS (
    SELECT *
    FROM ranking
    WHERE rk <= {top_n}
)
SELECT
    cli.IDENTIFICACIÓN AS id_cliente,
    cli.NOMBRE AS nombre_cliente,
    cli.TIPO_DOCUMENTO,
    cli.CLASIFICACION,
    cli."TIPO TARJETA" AS tipo_tarjeta,
    pref.NOMBRE_CATEGORIA AS categoria_preferida,
    pref.num_transacciones,
    pref.ultima_fecha,
    pref.rk AS nivel_preferencia
FROM preferencias_top_n pref
LEFT JOIN clientes cli
    ON pref.IDENTIFICACION = cli.IDENTIFICACIÓN
ORDER BY id_cliente, nivel_preferencia
"""

In [72]:
# Ejecutar la consulta y guardar el resultado
df_resultado = pd.read_sql_query(query, conn)

# Mostrar el resultado
df_resultado.head()

,id_cliente,nombre_cliente,TIPO_DOCUMENTO,CLASIFICACION,tipo_tarjeta,categoria_preferida,num_transacciones,ultima_fecha,nivel_preferencia
0,2A3B4C5D6E7F8G9H0,Laura López,DNI,Empresarial,Débito,None,1,2023-04-01 00:00:00,1
1,2A3B4C5D6E7F8G9H0,Santiago Vargas,Cédula,Personal,Crédito,None,1,2023-04-01 00:00:00,1
2,2A3B4C5D6E7F8G9H0,Laura López,DNI,Empresarial,Débito,Ropa,1,2021-11-15 00:00:00,2
3,2A3B4C5D6E7F8G9H0,Santiago Vargas,Cédula,Personal,Crédito,Ropa,1,2021-11-15 00:00:00,2
4,2A3B4C5D6E7F8G9H0,Laura López,DNI,Empresarial,Débito,Hogar,1,2021-06-15 00:00:00,3
